In [25]:
pip install -U transformers accelerate bitsandbytes

In [26]:
import os

In [27]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-01 12:18:03--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-01 12:18:03 (57.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [28]:
import requests
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [29]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [30]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [31]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [32]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

In [33]:

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

In [35]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)


In [36]:


messages = [
    {"role": "user", "content": "I just discovered the course. Can I still join?"},
]


generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


 I'm sorry, but it seems you've missed the opportunity to join the course. However, I can help you find similar courses or provide information on how to enroll in future sessions. Would you like me to assist you with that?


In [37]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
  messages = [
    {"role": "user", "content": prompt}
]
  generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

  output = pipe(messages, **generation_args)
  return output[0]['generated_text']


In [43]:
rag("I just discovered the course. Can I still join?")

' Yes, you can still join the course even if you discover it after the start date. You are still eligible to submit the homeworks, but remember to meet the deadlines for the final projects.'

In [44]:
rag("What is the video/zoom link to the stream for the “Office Hours” or live/workshop sessions? ")

" I'm sorry, but the CONTEXT provided does not include information about a video/zoom link for the “Office Hours” or live/workshop sessions. The details given are about the use of Youtube Live, Slido, and announcements on Telegram & Slack, as well as the recording availability of the sessions. If you need the specific link, please refer to the announcements channel or the DataTalksClub YouTube Channel."

In [42]:
rag(" I have registered for the [insert-zoomcamp-name]. When can I expect to receive the confirmation email?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


" I'm sorry, but the CONTEXT provided does not contain information about the confirmation email for the [insert-zoomcamp-name] course registration. The provided context discusses various issues and solutions related to Docker, Git, and GitHub CLI, as well as general course-related questions. If you have any specific questions about the confirmation email or the course, please let me know, and I'll do my best to assist you."